# PROJETO - Big Data para Dados Públicos
## Professor Maciel Calebe
## Análise de Propagação de Doenças  


### Arthur Boschini da Fonseca, Caio de Camargo Aranha Tieri, Guilherme Farias Caproni
##### Descrição:

Este projeto tem como objetivo empregar os dados dos registros de atendimentos médicos a fim de monitorar e antecipar a disseminação de doenças infecciosas, como a gripe ou a dengue, ao longo do tempo e em diversas áreas geográficas.


##### Database Utilizada
*DATASUS - SRAG 2019*: Disponível em: https://opendatasus.saude.gov.br/dataset/srag-2019

##### Dicionário de Dados
Disponível em: https://d26692udehoye.cloudfront.net/SRAG/2019/dicionario_de_dados_SRAG_hospitalizado_2019.pdf




In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('INFLUD19.csv', encoding='latin-1', on_bad_lines='skip', sep=';')

C:\Users\FONSECA CERTO\AppData\Local\Temp\ipykernel_7496\2263860273.py:4: DtypeWarning: Columns (96,104,106,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('INFLUD19.csv', encoding='latin-1', on_bad_lines='skip', sep=';')


In [15]:
df.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DT_DIGITA,HISTO_VGM,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PCR_SARS2,PAC_COCBO,PAC_DSCBO
0,10/01/2019,2,06/01/2019,2,MG,BELO HORIZONTE,1449.0,BELO HORIZONTE,310620,UNIDADE DE PRONTO ATENDIMENTO LESTE,...,14/03/2019,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,03/01/2019,1,01/01/2019,1,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSP MUN M BOI MIRIM,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02/01/2019,1,31/12/2018,1,PE,1,1497.0,RECIFE,261160,IMIP,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/01/2019,2,07/01/2019,2,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950,HOSPITAL MUNICIPAL DR MARIO GATTI CAMPINAS,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/01/2019,2,06/01/2019,2,PE,4,1499.0,CARUARU,260410,SECRETARIA MUNICIPAL DE SAUDE DE CARUARU,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Escolhendo colunas

In [31]:
informacoes_pessoais = df[["CS_SEXO", "DT_NASC", "CS_RACA", "SG_UF", "SURTO_SG", "NOSOCOMIAL", ]]
informacoes_pessoais.rename(columns={"CS_SEXO": "sexo", "DT_NASC": "data_nascimento", "CS_RACA": "raca", "SG_UF": "estado", "SURTO_SG": "surto_sindromegripal", "NOSOCOMIAL": "contraiu_no_hospital"}, inplace=True)
informacoes_pessoais.head()

C:\Users\FONSECA CERTO\AppData\Local\Temp\ipykernel_7496\540703258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  informacoes_pessoais.rename(columns={"CS_SEXO": "sexo", "DT_NASC": "data_nascimento", "CS_RACA": "raca", "SG_UF": "estado", "SURTO_SG": "surto_sindromegripal", "NOSOCOMIAL": "contraiu_no_hospital"}, inplace=True)


,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital
0,M,17/03/1988,1.0,MG,2.0,2.0
1,F,30/05/2018,1.0,SP,9.0,NaN
2,M,07/05/2017,4.0,PE,9.0,9.0
3,F,15/07/2018,1.0,SP,2.0,2.0
4,F,15/09/2018,4.0,PE,9.0,9.0


Trocando as legendas do dataframe para valores normais

In [40]:
informacoes_pessoais['raca'] = informacoes_pessoais['raca'].replace({1.0: 'Branca', 2.0: 'Preta', 3.0: 'Amarela', 4.0: 'Parda', 5.0: 'Indígena', 9.0: 'Ignorado'})
informacoes_pessoais['surto_sindromegripal'] = informacoes_pessoais['surto_sindromegripal'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
informacoes_pessoais['contraiu_no_hospital'] = informacoes_pessoais['contraiu_no_hospital'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado', None: 'Ignorado'})

C:\Users\FONSECA CERTO\AppData\Local\Temp\ipykernel_7496\2285531459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  informacoes_pessoais['raca'] = informacoes_pessoais['raca'].replace({1.0: 'Branca', 2.0: 'Preta', 3.0: 'Amarela', 4.0: 'Parda', 5.0: 'Indígena', 9.0: 'Ignorado'})
C:\Users\FONSECA CERTO\AppData\Local\Temp\ipykernel_7496\2285531459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  informacoes_pessoais['surto_sindromegripal'] = informacoes_pessoais['surto_sindromegripal'].replace({1.0: 'Si

In [41]:
informacoes_pessoais.head()

,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital
0,M,17/03/1988,Branca,MG,Não,Não
1,F,30/05/2018,Branca,SP,Ignorado,Ignorado
2,M,07/05/2017,Parda,PE,Ignorado,Ignorado
3,F,15/07/2018,Branca,SP,Não,Não
4,F,15/09/2018,Parda,PE,Ignorado,Ignorado


Informações de contágio

In [43]:
sintomas = df[["FEBRE", "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA", "VOMITO",  "OUTRO_SIN", "OUTRO_DES"]]

In [44]:
sintomas['FEBRE'] = sintomas['FEBRE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['TOSSE'] = sintomas['TOSSE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['GARGANTA'] = sintomas['GARGANTA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DISPNEIA'] = sintomas['DISPNEIA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DESC_RESP'] = sintomas['DESC_RESP'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['SATURACAO'] = sintomas['SATURACAO'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DIARREIA'] = sintomas['DIARREIA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['VOMITO'] = sintomas['VOMITO'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['OUTRO_SIN'] = sintomas['OUTRO_SIN'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})

C:\Users\FONSECA CERTO\AppData\Local\Temp\ipykernel_7496\3255288520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sintomas['FEBRE'] = sintomas['FEBRE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
C:\Users\FONSECA CERTO\AppData\Local\Temp\ipykernel_7496\3255288520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sintomas['TOSSE'] = sintomas['TOSSE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
C:\Users\FONSECA CERTO\AppData\Local\Temp\ipykernel_7496\3255288520.py:3: SettingWithCopyWarni

In [45]:
sintomas.head()

,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES
0,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Sim,DERRAME PLEURAL
1,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,NaN
2,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,NaN
3,Sim,Sim,Ignorado,Sim,Sim,Sim,Não,Não,Não,NaN
4,Sim,Não,Não,Sim,Sim,Sim,Ignorado,Ignorado,Não,NaN


Juntando as duas tabelas

In [48]:
#Juntando as duas tabelas: informacoes_pessoais e sintomas
df = pd.concat([informacoes_pessoais, sintomas], axis=1)
df.head()

,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES
0,M,17/03/1988,Branca,MG,Não,Não,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Sim,DERRAME PLEURAL
1,F,30/05/2018,Branca,SP,Ignorado,Ignorado,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,NaN
2,M,07/05/2017,Parda,PE,Ignorado,Ignorado,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,NaN
3,F,15/07/2018,Branca,SP,Não,Não,Sim,Sim,Ignorado,Sim,Sim,Sim,Não,Não,Não,NaN
4,F,15/09/2018,Parda,PE,Ignorado,Ignorado,Sim,Não,Não,Sim,Sim,Sim,Ignorado,Ignorado,Não,NaN
